In [1]:
import numpy as np
import random
from keras.optimizers import *


Using TensorFlow backend.
C:\Users\Nils\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Nils\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Nils\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ni

In [2]:
random.seed(2)

In [3]:
class Individual:
    
    def __init__(self, learn_rate, epochs, batch, neurons_dense, activation, optimizer):
        
        self.learn_rate = learn_rate
        self.epochs = epochs
        self.batch = batch          
        self.neurons_dense = neurons_dense
       
        self.activation = activation
        self.optimizer = optimizer
      
        self.loss = 0
        self.acc = 0
        self.prec = 0
        self.rec = 0
        self.f1 = 0
        self.generation = 0
        self.time = 0
        

In [4]:
class readGeneration:
    
    def __init__(self, learn_rate, epochs, batch, neurons_dense, activation, optimizer, loss, acc, 
                 prec, rec, f1, generation, time):
        
        self.learn_rate = learn_rate
        self.epochs = epochs
        self.batch = batch          
        self.neurons_dense = neurons_dense
       
        self.activation = activation
        
        self.optimizer = optimizer
      
        self.loss = loss
        self.acc = acc
        self.prec = prec
        self.rec = rec
        self.f1 = f1
        self.generation = generation
        self.time = time
    

In [5]:
def show (pop):
    for i in range(len(pop)):
        print("INDIVIDUUM:",i+1)
        print("Lernrate:",pop[i].learn_rate, "Epochen:",pop[i].epochs, "Batch:",pop[i].batch,)
        print("Aktivfkt:", pop[i].activation, "Optimizer:", pop[i].optimizer)
        print("Anzahl dense:", pop[i].neurons_dense)
        print("Acc:", pop[i].acc, "Loss:", pop[i].loss, "f1:", pop[i].f1, "Generation:", pop[i].generation, '\n \n')

In [6]:
def create_population(pop_count):
    
    pop =[]
    activation_list = ['elu', 'selu','relu','tanh','sigmoid', 'hard_sigmoid','exponential', 'linear', 
                    'softmax','softplus','softsign']
    optimizer_list = ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"]

    activation_list_pop = []
    optimizer_list_pop = []
  
    folds = int(pop_count/77)
    for j in range(0,folds) :   
        for k in range(len(activation_list)):
            for n in range(len(optimizer_list)):
                activation_list_pop.append(activation_list[k])
                optimizer_list_pop.append(optimizer_list[n])
            
    for i in range(0,pop_count):
        learn_rate    = round(random.uniform(0.00001,0.1),5)
        epochs        = random.randrange(1,64)
        batch         = random.randrange(1,512)        
    
        neurons_dense  = random.randrange(1,1200)
        
        activation  = activation_list_pop[i]
        optimizer = optimizer_list_pop[i]
                        
        pop.append(Individual(learn_rate, epochs, batch, neurons_dense, activation, optimizer))
        
    return pop

In [7]:
def select_fittest(pop):
    parents= []
    
    pop.sort(key = lambda x:x.acc, reverse = True)
    
    good_half = pop[:len(pop)//2]
       
    return good_half

In [8]:
def breed_new(pop, pop_count):
    children = []

    for i in range(0,pop_count):
        while True: 
            k = random.randrange(0,len(pop)//2) 
            l = random.randrange(0,len(pop))   
            if k != l:
                break
        parent_1 = pop[k]
        parent_2 = pop[l]
        
        child_lr = random.choice([parent_1.learn_rate, parent_2.learn_rate])
        child_epo = random.choice([parent_1.epochs, parent_2.epochs])
        child_bat = random.choice([parent_1.batch, parent_2.batch])        
        child_dense = random.choice([parent_1.neurons_dense, parent_2.neurons_dense])
        
        child_activation = random.choice([parent_1.activation, parent_2.activation])
        child_optimizer = random.choice([parent_1.optimizer, parent_2.optimizer])
    
        children.append(Individual(child_lr, child_epo, child_bat, 
                                   child_dense,
                                   child_activation, child_optimizer))   
    
    return children

In [9]:
def mutate_new(pop, max_mutation, mutation_rate):
    
    mutated = []
    
    activation_list = ['elu', 'selu','relu','tanh','sigmoid', 'hard_sigmoid','exponential', 'linear', 
                      'softmax','softplus','softsign']
    optimizer_list = ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"]
    
    mutation_flag = get_flag(len(pop),mutation_rate) 
    
    if max_mutation == 0:
    
        return pop
    else:
        for i in range(len(pop)):
            if mutation_flag[i] == 1:
               
                mutated_lr = round(pop[i].learn_rate + pop[i].learn_rate* round(random.uniform(-max_mutation,max_mutation),3),5)
                mutated_epochs = int(round(pop[i].epochs + pop[i].epochs*round(random.uniform(-max_mutation,max_mutation),3),0))
                mutated_batch = int(round(pop[i].batch + pop[i].batch*round(random.uniform(-max_mutation,max_mutation),3),0))
                
                mutated_neurons_dense = int(round(pop[i].neurons_dense + pop[i].neurons_dense*round(random.uniform(-max_mutation,max_mutation),3),0))
               
                nominal_mutation_flag = get_flag(1, mutation_rate) 
                
                if nominal_mutation_flag[0] == 1:
                    mutated_activation = random.choice(activation_list) 
                    mutated_optimizer =  random.choice(optimizer_list)
                else:
                    mutated_activation = pop[i].activation
                    mutated_optimizer = pop[i].optimizer
                
            
                mutated.append(Individual(mutated_lr, mutated_epochs, mutated_batch,
                                          mutated_neurons_dense,
                                          mutated_activation, mutated_optimizer))
            else:
                mutated.append(pop[i])
    return mutated

In [10]:
def check_boundaries(pop):
    for i in range(len(pop)):
        if not 0.00001 <= pop[i].learn_rate <= 0.1: 
            if pop[i].learn_rate > 0.05:
                pop[i].learn_rate = 0.1
            else:
                pop[i].learn_rate = 0.00001
            
        if not 1 <= pop[i].epochs <= 128:
            if pop[i].epochs > 100:
                pop[i].epochs = 128
            else:
                pop[i].epochs = 1
        
        if not 1 <= pop[i].batch <= 512:
            if pop[i].batch > 500:
                pop[i].batch = 512
            else:
                pop[i].batch = 1
        
        if not 1 <= pop[i].neurons_dense <= 1200:
            if pop[i].neurons_dense > 1000:
                pop[i].neurons_dense = 1200
            else:
                pop[i].neurons_dense = 1
                
    return pop

In [11]:
def create_optimizer(optimizer, learn_rate):
    if optimizer == 'SGD':
        rtr_optimizer = SGD(lr = learn_rate)
        
    if optimizer == 'RMSprop':
        rtr_optimizer = RMSprop(lr = learn_rate)
        
    if optimizer == 'Adagrad':
        rtr_optimizer = Adagrad(lr = learn_rate)
        
    if optimizer == 'Adadelta':
        rtr_optimizer = Adadelta(lr = learn_rate)
        
    if optimizer == 'Adam':
        rtr_optimizer = Adam(lr = learn_rate)
        
    if optimizer == 'Adamax':
        rtr_optimizer = Adamax(lr = learn_rate)
        
    if optimizer == 'Nadam':
        rtr_optimizer = Nadam(lr = learn_rate)
           
    return rtr_optimizer

In [12]:
def get_flag(length,mutation_rate):
    list_flag = []
    a = np.array([0,1])
    probability = np.array([1-mutation_rate,mutation_rate])
    for i in range(0,length):
        flag = np.random.choice(a,p= probability)
        list_flag.append(flag)
    return list_flag

In [13]:
def write_generation(pop, generation):
    for i in range(len(pop)): 
        pop[i].generation = generation
    return pop

In [14]:
def compare_max(pop, best):
    for i in range(len(pop)):
        if pop[i].acc > best.acc:
            best = pop[i]
    return best    

In [15]:
def write_to_file_jupyter (pop, filename):
    list_lr = []
    list_epochs = []
    list_batch =[]
    
    list_neurons_dense = []    
    
    list_activation = []
    list_optimizer = []
    
    list_loss = []
    list_acc = []
    list_prec = []
    list_rec = []
    list_f1 = []
    list_generation = []
    list_time = []
    
    
    for i in range(len(pop)):
        list_lr.append(pop[i].learn_rate)
        list_epochs.append(pop[i].epochs)
        list_batch.append(pop[i].batch)
        
        list_neurons_dense.append(pop[i].neurons_dense)
        
        list_activation.append(pop[i].activation)
                
        list_optimizer.append(pop[i].optimizer)
               
        list_loss.append(pop[i].loss)
        list_acc.append(pop[i].acc)
        list_prec.append(pop[i].prec)
        list_rec.append(pop[i].rec)
        list_f1.append(pop[i].f1)
        list_generation.append(pop[i].generation)
        list_time.append(pop[i].time)
        
        
    data_out = pd.DataFrame(list(zip(list_lr, list_epochs, list_batch,
                                     list_neurons_dense, list_activation, list_optimizer,
                                     list_loss, list_acc, list_prec, list_rec,
                                     list_f1, list_generation, list_time)))
    header = ["Learn_rate", "Epochs", "Batch",
              "Neurons_Dense", "Activation","Optimizer",
              "Loss","Accuracy", "Precision", "Recall", "F1", "Generation","Zeitstempel"]
    data_out.to_csv(filename, sep =';', header = header, decimal='.', float_format ='%.5f', mode = 'a')

In [16]:
def write_to_file_excel (pop, filename):
    list_lr = []
    list_epochs = []
    list_batch =[]
    
    list_neurons_dense = []    
    
    list_activation = []
    list_optimizer = []
    
    list_loss = []
    list_acc = []
    list_prec = []
    list_rec = []
    list_f1 = []
    list_generation = []
    list_time = []
    
    
    for i in range(len(pop)):
        list_lr.append(pop[i].learn_rate)
        list_epochs.append(pop[i].epochs)
        list_batch.append(pop[i].batch)
        
        list_neurons_dense.append(pop[i].neurons_dense)
        
        list_activation.append(pop[i].activation)
                
        list_optimizer.append(pop[i].optimizer)
               
        list_loss.append(pop[i].loss)
        list_acc.append(pop[i].acc)
        list_prec.append(pop[i].prec)
        list_rec.append(pop[i].rec)
        list_f1.append(pop[i].f1)
        list_generation.append(pop[i].generation)
        list_time.append(pop[i].time)
        
        
    data_out = pd.DataFrame(list(zip(list_lr, list_epochs, list_batch,
                                     list_neurons_dense, list_activation, list_optimizer,
                                     list_loss, list_acc, list_prec, list_rec,
                                     list_f1, list_generation, list_time)))
    header = ["Learn_rate", "Epochs", "Batch",
              "Neurons_Dense", "Activation","Optimizer",
              "Loss","Accuracy", "Precision", "Recall", "F1", "Generation","Zeitstempel"]
    data_out.to_csv(filename, sep =';', header = header, decimal=',', float_format ='%.5f', mode = 'a')

In [17]:
def read_from_file(filename):
    pop = []
    with open (filename, 'r') as f:
        read_data = csv.reader(f, delimiter =";")
        next(f)
        for line in f:
            splitted_line = line.split(';')
            learn_rate    = float(splitted_line[1])
            epochs        = int(splitted_line[2])
            batch         = int(splitted_line[3])
            neurons_dense = int(splitted_line[4])
            activation    = splitted_line[5]
            optimizer     = splitted_line[6]
            loss          = float(splitted_line[7])
            acc           = float(splitted_line[8])
            prec          = float(splitted_line[9])
            rec           = float(splitted_line[10])
            f1            = float(splitted_line[11])
            generation    = int(splitted_line[12])
            timestamp     = 0#float(splitted_line[13])

            pop.append(readGeneration(learn_rate, epochs, batch, neurons_dense, activation, optimizer, loss,
                                      acc, prec, rec, f1, generation, timestamp))
        return pop